In [1]:
pip install pytorch_lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 708.1/708.1 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 58.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 529.2/529.2 kB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.7/232.7 kB 31.7 MB/s eta 0:00:00
  Using cached tensorboard_data_server-0.6.1-py3-none-manylinux2010_x86_64.whl (4.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 49.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.3/93.3 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.4/123.4 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 61.6 MB/s eta 0:00:00:00:0100:01
  Using cached google_auth_oauthlib-0.4.6-py2.py3-none-any.whl (18 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 43.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.8/169.8 kB 28.7 MB/s eta 0:00:00
  Using cached p

In [2]:
pip install -U imbalanced-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.3/199.3 kB 16.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from torchmetrics import Recall
import joblib
from sklearn.preprocessing import MinMaxScaler    
from sklearn.model_selection import train_test_split
from pathlib import Path
from sklearn.preprocessing import PolynomialFeatures
from imblearn.over_sampling import SMOTE

In [2]:
df_train = pd.read_csv("/notebooks/Dataset/train_dataset_train.csv")

In [6]:
x, y = df_train[['Easting','Northing','Height','Reflectance']], df_train['Class']
sampling_strategy = {4: 300000, 1: 300000, 5: 300000, 64: 300000}
smote = SMOTE(sampling_strategy=sampling_strategy)
x_smote, y_smote = smote.fit_resample(x, y)

df_train = pd.concat((x_smote, y_smote), ignore_index=False, axis=1)

df_train['Easting_log'], df_train['Northing_log'], df_train['Height_log'], df_train['Reflectance_log'] = np.log10((df_train['Easting'], df_train['Northing'], df_train['Height'], (df_train['Reflectance']+45) ))

df_train = df_train.drop(['Easting', 'Northing', 'Height', 'Reflectance'], axis=1)

df_train = df_train[[col for col in df_train.columns if col != 'Class'] + ['Class']]

df_train.loc[df_train["Class"] == 64, "Class"] = 2

num_0 = len(df_train.loc[(df_train["Class"] == 0)])
num_1 = len(df_train.loc[(df_train["Class"] == 1)])
num_2 = len(df_train.loc[(df_train["Class"] == 2)])
num_3 = len(df_train.loc[(df_train["Class"] == 3)])
num_4 = len(df_train.loc[(df_train["Class"] == 4)])
num_5 = len(df_train.loc[(df_train["Class"] == 5)])

total_nums = num_0 + num_1 + num_2 + num_3 + num_4 + num_5

weight_0 = 1/(num_0/total_nums)/2
weight_1 = 1/(num_1/total_nums)/2
weight_2 = 1/(num_2/total_nums)/2
weight_3 = 1/(num_3/total_nums)/2
weight_4 = 1/(num_4/total_nums)/2
weight_5 = 1/(num_5/total_nums)/2

total_weight = np.array([weight_0, weight_1, weight_2, weight_3, weight_4, weight_5])

total_weight

In [30]:
total_weight = torch.from_numpy(total_weight).float()

In [31]:
X = df_train.iloc[:, 0:-1]
y = df_train.iloc[:, -1]

In [32]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [33]:
polier = PolynomialFeatures(3)
X_train = polier.fit_transform(X_train)
X_val = polier.transform(X_val)

scaler = MinMaxScaler(feature_range=(-1, 1))
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

X_train, y_train = np.array(X_train), np.array(y_train)
X_val, y_val = np.array(X_val), np.array(y_val)

In [34]:
joblib.dump(scaler, '/notebooks/Scalers/scaler_97.gz')
joblib.dump(polier, '/notebooks/Scalers/polier_97.gz')

['/notebooks/Scalers/polier_97.gz']

In [35]:
X_train.shape

(4175084, 35)

In [36]:
class ClassifierDataset(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)

train_dataset = ClassifierDataset(torch.from_numpy(X_train).float(), torch.from_numpy(y_train).long())
val_dataset = ClassifierDataset(torch.from_numpy(X_val).float(), torch.from_numpy(y_val).long())

In [37]:
EPOCHS = 15
BATCH_SIZE = 2048
LEARNING_RATE = 0.001
NUM_FEATURES = len(X_train[1])
NUM_CLASSES = 6

In [38]:
train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE)
val_loader = DataLoader(dataset=val_dataset, batch_size=BATCH_SIZE)

In [39]:
class SoftOrdering1DCNN(pl.LightningModule):

    def __init__(self, input_dim, output_dim, sign_size=32, cha_input=16, cha_hidden=32, 
                 K=2, dropout_input=0.2, dropout_hidden=0.2, dropout_output=0.2):
        super().__init__()

        hidden_size = sign_size*cha_input
        sign_size1 = sign_size
        sign_size2 = sign_size//2
        output_size = (sign_size//4) * cha_hidden

        self.hidden_size = hidden_size
        self.cha_input = cha_input
        self.cha_hidden = cha_hidden
        self.K = K
        self.sign_size1 = sign_size1
        self.sign_size2 = sign_size2
        self.output_size = output_size
        self.dropout_input = dropout_input
        self.dropout_hidden = dropout_hidden
        self.dropout_output = dropout_output

        self.batch_norm1 = nn.BatchNorm1d(input_dim)
        self.dropout1 = nn.Dropout(dropout_input)
        dense1 = nn.Linear(input_dim, hidden_size, bias=False)
        self.dense1 = nn.utils.weight_norm(dense1)

        # 1st conv layer
        self.batch_norm_c1 = nn.BatchNorm1d(cha_input)
        conv1 = conv1 = nn.Conv1d(
            cha_input, 
            cha_input*K, 
            kernel_size=5, 
            stride = 1, 
            padding=2,  
            groups=cha_input, 
            bias=False)
        self.conv1 = nn.utils.weight_norm(conv1, dim=None)

        self.ave_po_c1 = nn.AdaptiveAvgPool1d(output_size = sign_size2)

        # 2nd conv layer
        self.batch_norm_c2 = nn.BatchNorm1d(cha_input*K)
        self.dropout_c2 = nn.Dropout(dropout_hidden)
        conv2 = nn.Conv1d(
            cha_input*K, 
            cha_hidden, 
            kernel_size=3, 
            stride=1, 
            padding=1, 
            bias=False)
        self.conv2 = nn.utils.weight_norm(conv2, dim=None)

        # 3rd conv layer
        self.batch_norm_c3 = nn.BatchNorm1d(cha_hidden)
        self.dropout_c3 = nn.Dropout(dropout_hidden)
        conv3 = nn.Conv1d(
            cha_hidden, 
            cha_hidden, 
            kernel_size=3, 
            stride=1, 
            padding=1, 
            bias=False)
        self.conv3 = nn.utils.weight_norm(conv3, dim=None)
        

        # 4th conv layer
        self.batch_norm_c4 = nn.BatchNorm1d(cha_hidden)
        conv4 = nn.Conv1d(
            cha_hidden, 
            cha_hidden, 
            kernel_size=5, 
            stride=1, 
            padding=2, 
            groups=cha_hidden, 
            bias=False)
        self.conv4 = nn.utils.weight_norm(conv4, dim=None)

        self.avg_po_c4 = nn.AvgPool1d(kernel_size=4, stride=2, padding=1)

        self.flt = nn.Flatten()

        self.batch_norm2 = nn.BatchNorm1d(output_size)
        self.dropout2 = nn.Dropout(dropout_output)
        dense2 = nn.Linear(output_size, output_dim, bias=True)
        self.dense2 = nn.utils.weight_norm(dense2)        

        self.loss = nn.CrossEntropyLoss()

    def forward(self, x):
        x = self.batch_norm1(x)
        x = self.dropout1(x)
        x = nn.functional.celu(self.dense1(x))

        x = x.reshape(x.shape[0], self.cha_input, self.sign_size1)

        x = self.batch_norm_c1(x)
        x = nn.functional.relu(self.conv1(x))

        x = self.ave_po_c1(x)

        x = self.batch_norm_c2(x)
        x = self.dropout_c2(x)
        x = nn.functional.relu(self.conv2(x))
        x_s = x

        x = self.batch_norm_c3(x)
        x = self.dropout_c3(x)
        x = nn.functional.relu(self.conv3(x))

        x = self.batch_norm_c4(x)
        x = self.conv4(x)
        x =  x + x_s
        x = nn.functional.relu(x)

        x = self.avg_po_c4(x)

        x = self.flt(x)

        x = self.batch_norm2(x)
        x = self.dropout2(x)
        x = self.dense2(x)       

        return x

In [40]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [41]:
model = SoftOrdering1DCNN(
    input_dim=NUM_FEATURES, 
    output_dim=NUM_CLASSES, 
    sign_size=16, 
    cha_input=64, 
    cha_hidden=64, 
    K=2, 
    dropout_input=0.3, 
    dropout_hidden=0.3,
    dropout_output=0.4
)
# model = MulticlassClassification(num_feature = NUM_FEATURES, num_class=NUM_CLASSES)
model.to(device)

criterion = nn.CrossEntropyLoss(weight=total_weight).to(device)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
print(model)

SoftOrdering1DCNN(
  (batch_norm1): BatchNorm1d(35, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout1): Dropout(p=0.3, inplace=False)
  (dense1): Linear(in_features=35, out_features=1024, bias=False)
  (batch_norm_c1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv1): Conv1d(64, 128, kernel_size=(5,), stride=(1,), padding=(2,), groups=64, bias=False)
  (ave_po_c1): AdaptiveAvgPool1d(output_size=8)
  (batch_norm_c2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout_c2): Dropout(p=0.3, inplace=False)
  (conv2): Conv1d(128, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
  (batch_norm_c3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout_c3): Dropout(p=0.3, inplace=False)
  (conv3): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
  (batch_norm_c4): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, t

In [42]:
def multi_acc(y_pred, y_test):
    y_pred_softmax = torch.log_softmax(y_pred, dim = 1)
    _, y_pred_tags = torch.max(y_pred_softmax, dim = 1)    
      
    recall = Recall(average='macro', num_classes=6).to(device)    
    acc = recall(y_pred_tags, y_test)
    
    acc = torch.round(acc * 100)
    
    return acc

In [43]:
accuracy_stats = {
    'train': [],
    "val": []
}
loss_stats = {
    'train': [],
    "val": []
}

In [44]:
MODEL_SAVE_PATH = '/notebooks/Model/model_97.pth'

In [45]:
print("Begin training.")

model.train()
for epoch in range(1, EPOCHS+1):    
    with tqdm(train_loader, unit="batch") as tepoch:
        train_epoch_loss = 0
        train_epoch_acc = 0
        for X_train_batch, y_train_batch in tepoch:
            tepoch.set_description(f"Epoch {epoch}")
            X_train_batch, y_train_batch = X_train_batch.to(device), y_train_batch.to(device)
            optimizer.zero_grad()

            y_train_pred = model(X_train_batch)

            train_loss = criterion(y_train_pred, y_train_batch)
            train_acc = multi_acc(y_train_pred, y_train_batch)

            train_loss.backward()
            optimizer.step()
            

            train_epoch_loss += train_loss.item()
            train_epoch_acc += train_acc.item()
            # tepoch.set_postfix(loss=train_loss.item(), accuracy=train_acc.item())
            
        with torch.inference_mode():

            val_epoch_loss = 0
            val_epoch_acc = 0

            model.eval()
            for X_val_batch, y_val_batch in val_loader:
                X_val_batch, y_val_batch = X_val_batch.to(device), y_val_batch.to(device)

                y_val_pred = model(X_val_batch)

                val_loss = criterion(y_val_pred, y_val_batch)
                val_acc = multi_acc(y_val_pred, y_val_batch)

                val_epoch_loss += val_loss.item()
                val_epoch_acc += val_acc.item()
                
        if val_loss < val_epoch_loss:
            print(f"Saving model to: {MODEL_SAVE_PATH}")
            torch.save(obj=model.state_dict(), f=MODEL_SAVE_PATH)
            
        loss_stats['train'].append(train_epoch_loss/len(train_loader))
        loss_stats['val'].append(val_epoch_loss/len(val_loader))
        accuracy_stats['train'].append(train_epoch_acc/len(train_loader))
        accuracy_stats['val'].append(val_epoch_acc/len(val_loader))
    
        if epoch % 1 == 0:
            print(f'Epoch {epoch+0:03}: | Train Loss: {train_epoch_loss/len(train_loader):.5f} | Val Loss: {val_epoch_loss/len(val_loader):.5f} | Train Recall:{train_epoch_acc/len(train_loader):.3f}| Val Recall:{val_epoch_acc/len(val_loader):.3f}')


Begin training.


  0%|          | 0/2039 [00:00<?, ?batch/s]

Saving model to: /notebooks/Model/model_97.pth
Epoch 001: | Train Loss: 0.46727 | Val Loss: 0.82882 | Train Recall:82.674| Val Recall:72.498


  0%|          | 0/2039 [00:00<?, ?batch/s]

Saving model to: /notebooks/Model/model_97.pth
Epoch 002: | Train Loss: 0.10946 | Val Loss: 0.08172 | Train Recall:96.010| Val Recall:97.045


...

Saving model to: /notebooks/Model/model_97.pth
Epoch 013: | Train Loss: 0.05300 | Val Loss: 0.05114 | Train Recall:98.048| Val Recall:98.124


  0%|          | 0/2039 [00:00<?, ?batch/s]

Saving model to: /notebooks/Model/model_97.pth
Epoch 014: | Train Loss: 0.05236 | Val Loss: 0.04975 | Train Recall:98.062| Val Recall:98.159


  0%|          | 0/2039 [00:00<?, ?batch/s]

Saving model to: /notebooks/Model/model_97.pth
Epoch 015: | Train Loss: 0.05166 | Val Loss: 0.04896 | Train Recall:98.083| Val Recall:98.210


In [32]:
MODEL_LOAD_PATH = '/notebooks/Model/model_97.pth'

In [33]:
model_loaded = SoftOrdering1DCNN(
    input_dim=NUM_FEATURES, 
    output_dim=NUM_CLASSES, 
    sign_size=16, 
    cha_input=64, 
    cha_hidden=64, 
    K=2, 
    dropout_input=0.1, 
    dropout_hidden=0.1,
    dropout_output=0.5
)
model_loaded.load_state_dict(torch.load(f=MODEL_LOAD_PATH))
model_loaded.to(device)

SoftOrdering1DCNN(
  (batch_norm1): BatchNorm1d(35, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout1): Dropout(p=0.3, inplace=False)
  (dense1): Linear(in_features=35, out_features=1024, bias=False)
  (batch_norm_c1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv1): Conv1d(64, 128, kernel_size=(5,), stride=(1,), padding=(2,), groups=64, bias=False)
  (ave_po_c1): AdaptiveAvgPool1d(output_size=8)
  (batch_norm_c2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout_c2): Dropout(p=0.3, inplace=False)
  (conv2): Conv1d(128, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
  (batch_norm_c3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout_c3): Dropout(p=0.3, inplace=False)
  (conv3): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
  (batch_norm_c4): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, t

In [46]:
df_test_raw = pd.read_csv("/notebooks/Dataset/test_dataset_test.csv")
# df_test_raw = df_test_raw.drop('id', axis=1)

In [47]:
df_test_raw['Easting_log'], df_test_raw['Northing_log'], df_test_raw['Height_log'], df_test_raw['Reflectance_log'] = np.log10((df_test_raw['Easting'], df_test_raw['Northing'], 
                                                                                                                   df_test_raw['Height'], (df_test_raw['Reflectance']+45) ))

df_test_raw = df_test_raw.drop(['id', 'Easting', 'Northing', 'Height', 'Reflectance'], axis=1)

In [38]:
polier = joblib.load('/notebooks/Scalers/polier_97.gz')
scaler = joblib.load('/notebooks/Scalers/scaler_97.gz')

In [48]:
df_test = polier.transform(df_test_raw)
df_test = scaler.transform(df_test)

class ClassifierDatasetTest(Dataset):
    
    def __init__(self, X_data):
        self.X_data = X_data       
        
    def __getitem__(self, index):
        return self.X_data[index]
        
    def __len__ (self):
        return len(self.X_data)

test_dataset = ClassifierDatasetTest(torch.from_numpy(df_test).float())

test_loader = DataLoader(dataset=test_dataset, batch_size=64)

y_pred_list = []
with torch.inference_mode():
    model.eval()
    for X_batch in test_loader:
        X_batch = X_batch.to(device)
        y_test_pred = model(X_batch)        
        _, y_pred_tags = torch.max(y_test_pred, dim = 1)        
        y_pred_list.append(y_pred_tags.squeeze().cpu().numpy())

In [49]:
import itertools
y_pred_list1 = [a.squeeze().tolist() for a in y_pred_list]
y_pred_list2 = list(itertools.chain.from_iterable(y_pred_list1))
cols = ['id']
df_test1 = pd.read_csv("/notebooks/Dataset/test_dataset_test.csv", usecols=cols)
df_test1['Class'] = [a for a in y_pred_list2]
df_test1.loc[df_test1["Class"] == 2, "Class"] = 64

In [50]:
df_test1.to_csv('/notebooks/Dataset/test_check_97.csv', index=False)
df_test1['Class'].value_counts()

0     1143997
3      555846
4       55560
5       23742
1       17792
64      12532
Name: Class, dtype: int64

In [51]:
df_test_add = pd.read_csv("/notebooks/Dataset/test_check_77.csv")

In [55]:
df_test_add['Class'].value_counts()

0     1151008
3      552889
4       53300
5       23757
1       17362
64      11153
Name: Class, dtype: int64